In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split
X_data=pd.read_csv("A:\\Coding\\ML\\Kaggle\\House-price-prediction-master\\train.csv",index_col='Id')
X_test=pd.read_csv("A:\\Coding\\ML\\Kaggle\\House-price-prediction-master\\test.csv",index_col='Id')
X_data.dropna(axis=0,subset=['SalePrice'],inplace=True)
y=X_data.SalePrice
X_data.drop(['SalePrice'],axis=1,inplace=True)
X=X_data.select_dtypes(exclude=['object'])
X_test=X_test.select_dtypes(exclude=['object'])
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)
print(X_train)


      MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                            
619           20         90.0    11694            9            5       2007   
871           20         60.0     6600            5            5       1962   
93            30         80.0    13360            5            7       1921   
818           20          NaN    13265            8            5       2002   
303           20        118.0    13704            7            5       2001   
...          ...          ...      ...          ...          ...        ...   
764           60         82.0     9430            8            5       1999   
836           20         60.0     9600            4            7       1950   
1217          90         68.0     8930            6            5       1978   
560          120          NaN     3196            7            5       2003   
685           60         58.0    16770            7 

In [4]:

# seeing missing data 
X_see=X_train.isnull().sum()
print(X_see[X_see>0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def Score_cal(X_train,X_valid,y_train,y_valid):
    model=RandomForestRegressor(n_estimators=100,random_state=0)
    model.fit(X_train,y_train)
    pred=model.predict(X_valid)
    return mean_absolute_error(y_valid,pred)
    
    

In [6]:
#score by removing cols with missing values
removing_cols=[col for col in X_train.columns if X_train[col].isnull().any()]
X_train_reduce=X_train.drop(removing_cols,axis=1)
X_valid_reduce=X_valid.drop(removing_cols,axis=1)
print(Score_cal(X_train_reduce,X_valid_reduce,y_train,y_valid))


17837.82570776256


In [7]:
#score by imputation 
from sklearn.impute import SimpleImputer
imputer=SimpleImputer()
X_train_imputed=pd.DataFrame(imputer.fit_transform(X_train))
X_valid_imputed=pd.DataFrame(imputer.transform(X_valid))

X_train_imputed.columns=X_train.columns
X_valid_imputed.columns=X_valid.columns
print(Score_cal(X_train_imputed,X_valid_imputed,y_train,y_valid))


18062.894611872147


In [8]:
# score by imputed median 
imputer=SimpleImputer(strategy='median')
X_train_imputed=pd.DataFrame(imputer.fit_transform(X_train))
X_valid_imputed=pd.DataFrame(imputer.transform(X_valid))

X_train_imputed.columns=X_train.columns
X_valid_imputed.columns=X_valid.columns
print(Score_cal(X_train_imputed,X_valid_imputed,y_train,y_valid))
X_test_final=pd.DataFrame(imputer.transform(X_test))
model=RandomForestRegressor(n_estimators=100,random_state=0)
model.fit(X_train_imputed,y_train)
pred=model.predict(X_test_final)
final_pred=model.predict(X_test_final)

17791.59899543379


In [9]:
output=pd.DataFrame({'Id': X_test.index,'SalePrice': final_pred})
output.to_csv("Final_submission.csv",index=False)